In [1]:
from utils.db_util import PostgreSQLDatabase
from utils.api_reader import SigmaWalletReader
from datetime import datetime
import pandas as pd

In [2]:
def format_datetime(time_str):
    # List of possible datetime formats to try
    formats = [
        "%Y-%m-%dT%H:%M:%S.%fZ",  # datetime with microseconds and 'Z' timezone
        "%Y-%m-%dT%H:%M:%S",      # datetime without microseconds or timezone
        "%Y-%m-%dT%H:%M:%S%z",    # datetime with timezone offset
        "%Y-%m-%dT%H:%M:%SZ"      # datetime with 'Z' but without microseconds
    ]
    
    # Attempt to parse the datetime string with each format
    for fmt in formats:
        try:
            dt = datetime.strptime(time_str, fmt)
            return dt.strftime("%Y-%m-%d %H:%M:%S")  # Converts to your desired format
        except ValueError:
            continue  # Try the next format if the current one fails
    
    # If no format matches, raise an exception
    raise ValueError(f"Time data '{time_str}' does not match any expected format")

In [3]:
database = PostgreSQLDatabase('marctheshark', 'password', 'localhost', 5432, 'mining-db')
database.connect()
database.get_cursor()

<cursor object at 0x7609e0539030; closed: 0>

In [4]:
reader = SigmaWalletReader(config_path="../conf")
reader.update_data()

In [5]:
url = '{}/{}'.format(reader.base_api, 'blocks?pageSize=5000')
block_data = reader.get_api_data(url)
block_df = pd.DataFrame(block_data)
# block_df['rolling_effort'] =block_df['effort'].expanding().mean()
block_df = block_df.rename(columns={'created': 'time_found'})
block_df['time_found'] = [format_datetime(data) for data in block_df.time_found]
data = reader.data

# block_df = block_df.rename(columns={'Time Found': 'Time_Found', 'Rolling Effort' : 'Rolling_Effort',
                                    # 'Confirmation [%]': 'Confirmation', 'reward [erg]': 'reward_erg', 'effort [%]': 'effort_percentage'})

In [6]:
block_data[0]

{'poolId': 'ErgoSigmanauts',
 'blockHeight': 1252559,
 'networkDifficulty': 422881.6589500178,
 'status': 'pending',
 'confirmationProgress': 0.4722222222222222,
 'effort': 0.017699607118926087,
 'transactionConfirmationData': '56dcc160776a8a71',
 'reward': 27.0221,
 'infoLink': 'https://explorer.ergoplatform.com/en/blocks/f12b63d2743427539774bdc9f76bb207fa7a82b659cfa4a7a4ce7fecc67517ea',
 'hash': 'f12b63d2743427539774bdc9f76bb207fa7a82b659cfa4a7a4ce7fecc67517ea',
 'miner': '9hSk13XDU8keWAp3KxKSgUVCkfDcw577jAjM6SgbRpuYeL8ygPH',
 'source': 'ErgoSigmanauts',
 'created': '2024-04-28T03:06:52.686688Z'}

In [7]:
block_df.head()

,poolId,blockHeight,networkDifficulty,status,confirmationProgress,effort,transactionConfirmationData,reward,infoLink,hash,miner,source,time_found
0,ErgoSigmanauts,1252559,422881.658950,pending,0.472222,0.017700,56dcc160776a8a71,27.0221,https://explorer.ergoplatform.com/en/blocks/f1...,f12b63d2743427539774bdc9f76bb207fa7a82b659cfa4...,9hSk13XDU8keWAp3KxKSgUVCkfDcw577jAjM6SgbRpuYeL...,ErgoSigmanauts,2024-04-28 03:06:52
1,ErgoSigmanauts,1252553,422881.658950,pending,0.555556,1.856882,50230ed620a8fa0b,27.0000,https://explorer.ergoplatform.com/en/blocks/9d...,9d7544bf89b699e7d31eaf49c4b8610d62d6545c1de9d2...,9i3P4Ah9jp7nnSNMg6gxjXXHEWjkkBbPJR5RLRkMwEt3DZ...,ErgoSigmanauts,2024-04-28 02:55:03
2,ErgoSigmanauts,1251978,380358.285885,confirmed,1.000000,0.112525,50a3000448228642,27.0010,https://explorer.ergoplatform.com/en/blocks/58...,58be0effcc39dc7fc6bbc745acfd627ff3e30eab98668c...,9i8wsL9HYe4wRtuxXtnvki31uGJd6avKoQ79BXbz2sHWNZ...,ErgoSigmanauts,2024-04-27 07:59:45
3,ErgoSigmanauts,1251939,366775.443497,confirmed,1.000000,0.244779,502468dac3622052,27.0182,https://explorer.ergoplatform.com/en/blocks/a4...,a4dc120a4aa17097a163160eeb41c3dba31780f4086491...,9i3P4Ah9jp7nnSNMg6gxjXXHEWjkkBbPJR5RLRkMwEt3DZ...,ErgoSigmanauts,2024-04-27 06:54:45
4,ErgoSigmanauts,1251871,366775.443497,confirmed,1.000000,1.039931,50d8fd9c81187de2,27.0088,https://explorer.ergoplatform.com/en/blocks/2b...,2be39f033f12f55638a0e9a8755b7f96b73d15b948990c...,9fxwXaXp7x3i8gzfZ5zqMJJK1DbtyUeVGQL3mBaCg2cB8y...,ErgoSigmanauts,2024-04-27 04:31:16


In [8]:
block_df.columns

Index(['poolId', 'blockHeight', 'networkDifficulty', 'status',
       'confirmationProgress', 'effort', 'transactionConfirmationData',
       'reward', 'infoLink', 'hash', 'miner', 'source', 'time_found'],
      dtype='object')

In [9]:
block_cols = ['poolId VARCHAR(255)',
              'blockHeight INTEGER',
              'networkDifficulty NUMERIC',
              'status VARCHAR(255)',
              'confirmationProgress NUMERIC',
              'effort NUMERIC',
              'transactionConfirmationData VARCHAR(255)',
              'reward NUMERIC', 
              'infoLink VARCHAR(255)',
              'hash VARCHAR(255)', 
              'miner VARCHAR(255)',
              'source VARCHAR(255)',
              'time_found TIMESTAMP',]

In [10]:
data_cols = [
    'fee NUMERIC',  # Numeric type for precision
    'paid NUMERIC',  # Numeric type for decimal values
    'blocks INTEGER',  # Integer value for block counts
    'last_block_found TIMESTAMP',  # Timestamp for dates, assuming proper conversion before storage
    'enabled BOOLEAN',  # Boolean type for true/false
    'minimumPayment NUMERIC',  # Numeric type for precision of payments
    'payoutScheme VARCHAR(255)',  # String type for defined payout schemes
    'connectedMiners INTEGER',  # Integer for counting connected miners
    'poolHashrate NUMERIC',  # Numeric type for decimal values
    'sharesPerSecond NUMERIC',  # Numeric type for decimal values
    'networkType VARCHAR(50)',  # String type for network types
    'networkHashrate NUMERIC',  # Numeric type for hash rates
    'networkDifficulty NUMERIC',  # Numeric type for network difficulty
    'lastNetworkBlockTime TIMESTAMP',  # Timestamp for block times
    'blockHeight INTEGER',  # Integer for block height values
    'connectedPeers INTEGER',  # Integer for counting connected peers
    'rewardType VARCHAR(50)',  # String type for reward types
    'poolEffort NUMERIC',  # Numeric type for pool effort
    'poolTTF NUMERIC',  # Numeric type for pool time to find
    # 'db_timestamp TIMESTAMP'  # Timestamp for the exact time data was recorded
]

In [11]:
# data['db_timestamp'] = datetime.datetime.now()
del data['payoutSchemeConfig']
del data['extra']
data

{'fee': 0.9,
 'paid': 1451.190902122231,
 'blocks': 55,
 'last_block_found': 'Sunday, April 28, 2024 at 03:06:52 AM',
 'enabled': True,
 'minimumPayment': 0.5,
 'payoutScheme': 'PPLNS',
 'connectedMiners': 36,
 'poolHashrate': 54.27,
 'sharesPerSecond': 4,
 'networkType': 'mainnet',
 'networkHashrate': 15.135751082257066,
 'networkDifficulty': 1.816290129870848,
 'lastNetworkBlockTime': '2024-04-28T04:19:33.7752039Z',
 'blockHeight': 1252595,
 'connectedPeers': 119,
 'rewardType': 'POW',
 'poolEffort': 13.429,
 'poolTTF': 0.387}

In [12]:
database.delete_table('data')
database.delete_table('block')

database.create_table('data', data_cols)
database.create_table('block', block_cols)

database.insert_data('data', data)

Table data deleted successfully.
Table block deleted successfully.


In [13]:
for index, row in block_df.iterrows():
    database.insert_data('block', row.to_dict())


In [14]:
sample = database.fetch_data('block')
sample = sample.sort_values('blockheight', ascending=False)
sample

/home/whaleshark/Documents/sigmanaut-mining-pool-ui/utils/db_util.py:133: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, self.conn)


,poolid,blockheight,networkdifficulty,status,confirmationprogress,effort,transactionconfirmationdata,reward,infolink,hash,miner,source,time_found
0,ErgoSigmanauts,1252559,422881.658950,pending,0.472222,0.017700,56dcc160776a8a71,27.022100,https://explorer.ergoplatform.com/en/blocks/f1...,f12b63d2743427539774bdc9f76bb207fa7a82b659cfa4...,9hSk13XDU8keWAp3KxKSgUVCkfDcw577jAjM6SgbRpuYeL...,ErgoSigmanauts,2024-04-28 03:06:52
1,ErgoSigmanauts,1252553,422881.658950,pending,0.555556,1.856882,50230ed620a8fa0b,27.000000,https://explorer.ergoplatform.com/en/blocks/9d...,9d7544bf89b699e7d31eaf49c4b8610d62d6545c1de9d2...,9i3P4Ah9jp7nnSNMg6gxjXXHEWjkkBbPJR5RLRkMwEt3DZ...,ErgoSigmanauts,2024-04-28 02:55:03
2,ErgoSigmanauts,1251978,380358.285885,confirmed,1.000000,0.112525,50a3000448228642,27.001000,https://explorer.ergoplatform.com/en/blocks/58...,58be0effcc39dc7fc6bbc745acfd627ff3e30eab98668c...,9i8wsL9HYe4wRtuxXtnvki31uGJd6avKoQ79BXbz2sHWNZ...,ErgoSigmanauts,2024-04-27 07:59:45
3,ErgoSigmanauts,1251939,366775.443497,confirmed,1.000000,0.244779,502468dac3622052,27.018200,https://explorer.ergoplatform.com/en/blocks/a4...,a4dc120a4aa17097a163160eeb41c3dba31780f4086491...,9i3P4Ah9jp7nnSNMg6gxjXXHEWjkkBbPJR5RLRkMwEt3DZ...,ErgoSigmanauts,2024-04-27 06:54:45
4,ErgoSigmanauts,1251871,366775.443497,confirmed,1.000000,1.039931,50d8fd9c81187de2,27.008800,https://explorer.ergoplatform.com/en/blocks/2b...,2be39f033f12f55638a0e9a8755b7f96b73d15b948990c...,9fxwXaXp7x3i8gzfZ5zqMJJK1DbtyUeVGQL3mBaCg2cB8y...,ErgoSigmanauts,2024-04-27 04:31:16
5,ErgoSigmanauts,1251567,388383.156547,confirmed,1.000000,0.261117,555f000b343364e4,27.005100,https://explorer.ergoplatform.com/en/blocks/f8...,f826ef482455161ca689de707f3c239b3ca8a6e7360a2c...,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...,ErgoSigmanauts,2024-04-26 18:24:58
6,ErgoSigmanauts,1251499,388383.156547,confirmed,1.000000,6.396079,507971cae2ee4a18,27.004200,https://explorer.ergoplatform.com/en/blocks/c7...,c7c63f408c339ae4482914cbb8e93ad48b17defd20ec90...,9fHLdYBz5fPdDfMkGMfAsuj6EfXCpmM4GkamT23xeT3hzd...,ErgoSigmanauts,2024-04-26 16:00:40
7,ErgoSigmanauts,1249527,315887.249576,confirmed,1.000000,0.341527,50164a42c64cc4f1,27.015100,https://explorer.ergoplatform.com/en/blocks/4f...,4fea81e1a1a823f1c395a7d0757208381c9f6da4f68b83...,9fRQ8iCiCZekAewtjHmu7AmucWjvBTNUMeBFtnNkpfyWsN...,ErgoSigmanauts,2024-04-23 22:19:10
8,ErgoSigmanauts,1249440,315887.249576,confirmed,1.000000,0.019216,50187b62b18da535,27.010000,https://explorer.ergoplatform.com/en/blocks/22...,224c94abcd0ffa48553f6988c34bcb92c2490e76861f1c...,9f3FRr4XudxVs1V35At1X5yj7LmQmnWqG46LqFKVNRf2Tu...,ErgoSigmanauts,2024-04-23 19:17:03
9,ErgoSigmanauts,1249432,315887.249576,confirmed,1.000000,0.639227,5057d49b44b4ed21,27.012200,https://explorer.ergoplatform.com/en/blocks/ac...,acb0c18d072bc0fc0d0e69e95cb07ba996b53ef0ac669a...,9eZVqXVnrVWQKK19b7E7kp4ZyNqanp2z1mpKUJRaouNsme...,ErgoSigmanauts,2024-04-23 19:06:49


In [15]:
new_block_data = {
    'poolId': 'pool123',
    'blockHeight': 672835,
    'networkDifficulty': 19.963,
    'status': 'unconfirmed',
    'confirmationProgress': 0.7,  # This is a critical field for the logic.
    'effort': 120.5,
    'transactionConfirmationData': 'tx12345',
    'reward': 6.25,
    'infoLink': 'http://example.com/info',
    'hash': 'hash_value_123456',  # Unique identifier
    'miner': 'miner_name',
    'source': 'source_info',
    'created': '2022-01-01T12:00:00',
    # 'rolling_effort': 10,
}
# database.update_or_insert('block', new_block_data)
# database.fetch_data('block')

In [16]:
block_data.append(new_block_data)
for data in block_data:
            # data['rolling_effort'] = data['effort'].expanding().mean()
            data['time_found'] = format_datetime(data.pop('created'))
            data['confirmationProgress'] = data['confirmationProgress'] * 100
            data['networkDifficulty'] = round(data['networkDifficulty'], 2)
            data['effort'] = round(data['effort'], 2)
            data['reward'] = round(data['reward'], 2)
            data['miner'] = '{}...{}'.format(data['miner'][:3], data['miner'][-5:])
            # database.update_or_insert('block', data)

database.fetch_data('block')

/home/whaleshark/Documents/sigmanaut-mining-pool-ui/utils/db_util.py:133: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, self.conn)


,poolid,blockheight,networkdifficulty,status,confirmationprogress,effort,transactionconfirmationdata,reward,infolink,hash,miner,source,time_found
0,ErgoSigmanauts,1252559,422881.658950,pending,0.472222,0.017700,56dcc160776a8a71,27.022100,https://explorer.ergoplatform.com/en/blocks/f1...,f12b63d2743427539774bdc9f76bb207fa7a82b659cfa4...,9hSk13XDU8keWAp3KxKSgUVCkfDcw577jAjM6SgbRpuYeL...,ErgoSigmanauts,2024-04-28 03:06:52
1,ErgoSigmanauts,1252553,422881.658950,pending,0.555556,1.856882,50230ed620a8fa0b,27.000000,https://explorer.ergoplatform.com/en/blocks/9d...,9d7544bf89b699e7d31eaf49c4b8610d62d6545c1de9d2...,9i3P4Ah9jp7nnSNMg6gxjXXHEWjkkBbPJR5RLRkMwEt3DZ...,ErgoSigmanauts,2024-04-28 02:55:03
2,ErgoSigmanauts,1251978,380358.285885,confirmed,1.000000,0.112525,50a3000448228642,27.001000,https://explorer.ergoplatform.com/en/blocks/58...,58be0effcc39dc7fc6bbc745acfd627ff3e30eab98668c...,9i8wsL9HYe4wRtuxXtnvki31uGJd6avKoQ79BXbz2sHWNZ...,ErgoSigmanauts,2024-04-27 07:59:45
3,ErgoSigmanauts,1251939,366775.443497,confirmed,1.000000,0.244779,502468dac3622052,27.018200,https://explorer.ergoplatform.com/en/blocks/a4...,a4dc120a4aa17097a163160eeb41c3dba31780f4086491...,9i3P4Ah9jp7nnSNMg6gxjXXHEWjkkBbPJR5RLRkMwEt3DZ...,ErgoSigmanauts,2024-04-27 06:54:45
4,ErgoSigmanauts,1251871,366775.443497,confirmed,1.000000,1.039931,50d8fd9c81187de2,27.008800,https://explorer.ergoplatform.com/en/blocks/2b...,2be39f033f12f55638a0e9a8755b7f96b73d15b948990c...,9fxwXaXp7x3i8gzfZ5zqMJJK1DbtyUeVGQL3mBaCg2cB8y...,ErgoSigmanauts,2024-04-27 04:31:16
5,ErgoSigmanauts,1251567,388383.156547,confirmed,1.000000,0.261117,555f000b343364e4,27.005100,https://explorer.ergoplatform.com/en/blocks/f8...,f826ef482455161ca689de707f3c239b3ca8a6e7360a2c...,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...,ErgoSigmanauts,2024-04-26 18:24:58
6,ErgoSigmanauts,1251499,388383.156547,confirmed,1.000000,6.396079,507971cae2ee4a18,27.004200,https://explorer.ergoplatform.com/en/blocks/c7...,c7c63f408c339ae4482914cbb8e93ad48b17defd20ec90...,9fHLdYBz5fPdDfMkGMfAsuj6EfXCpmM4GkamT23xeT3hzd...,ErgoSigmanauts,2024-04-26 16:00:40
7,ErgoSigmanauts,1249527,315887.249576,confirmed,1.000000,0.341527,50164a42c64cc4f1,27.015100,https://explorer.ergoplatform.com/en/blocks/4f...,4fea81e1a1a823f1c395a7d0757208381c9f6da4f68b83...,9fRQ8iCiCZekAewtjHmu7AmucWjvBTNUMeBFtnNkpfyWsN...,ErgoSigmanauts,2024-04-23 22:19:10
8,ErgoSigmanauts,1249440,315887.249576,confirmed,1.000000,0.019216,50187b62b18da535,27.010000,https://explorer.ergoplatform.com/en/blocks/22...,224c94abcd0ffa48553f6988c34bcb92c2490e76861f1c...,9f3FRr4XudxVs1V35At1X5yj7LmQmnWqG46LqFKVNRf2Tu...,ErgoSigmanauts,2024-04-23 19:17:03
9,ErgoSigmanauts,1249432,315887.249576,confirmed,1.000000,0.639227,5057d49b44b4ed21,27.012200,https://explorer.ergoplatform.com/en/blocks/ac...,acb0c18d072bc0fc0d0e69e95cb07ba996b53ef0ac669a...,9eZVqXVnrVWQKK19b7E7kp4ZyNqanp2z1mpKUJRaouNsme...,ErgoSigmanauts,2024-04-23 19:06:49


In [17]:
data = {
    'poolId': 'pool123',
    'blockHeight': 672835,
    'networkDifficulty': 19.963,
    'status': 'confirmed',
    'confirmationProgress': 1,  # This is a critical field for the logic.
    'effort': 120.5,
    'transactionConfirmationData': 'tx12345',
    'reward': 6.25,
    'infoLink': 'http://example.com/info',
    'hash': 'hash_value_123456',  # Unique identifier
    'miner': 'miner_name',
    'source': 'source_info',
    'created': '2022-01-01T12:00:00'
}
# block_data.append(new_block_data_2)
# for data in block_data:
            # data['rolling_effort'] = data['effort'].expanding().mean()
data['time_found'] = format_datetime(data.pop('created'))
data['confirmationProgress'] = data['confirmationProgress'] * 100
data['networkDifficulty'] = round(data['networkDifficulty'], 2)
data['effort'] = round(data['effort'], 2)
data['reward'] = round(data['reward'], 2)
data['miner'] = '{}...{}'.format(data['miner'][:3], data['miner'][-5:])
# database.update_or_insert('block', data)

database.fetch_data('block')

/home/whaleshark/Documents/sigmanaut-mining-pool-ui/utils/db_util.py:133: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, self.conn)


,poolid,blockheight,networkdifficulty,status,confirmationprogress,effort,transactionconfirmationdata,reward,infolink,hash,miner,source,time_found
0,ErgoSigmanauts,1252559,422881.658950,pending,0.472222,0.017700,56dcc160776a8a71,27.022100,https://explorer.ergoplatform.com/en/blocks/f1...,f12b63d2743427539774bdc9f76bb207fa7a82b659cfa4...,9hSk13XDU8keWAp3KxKSgUVCkfDcw577jAjM6SgbRpuYeL...,ErgoSigmanauts,2024-04-28 03:06:52
1,ErgoSigmanauts,1252553,422881.658950,pending,0.555556,1.856882,50230ed620a8fa0b,27.000000,https://explorer.ergoplatform.com/en/blocks/9d...,9d7544bf89b699e7d31eaf49c4b8610d62d6545c1de9d2...,9i3P4Ah9jp7nnSNMg6gxjXXHEWjkkBbPJR5RLRkMwEt3DZ...,ErgoSigmanauts,2024-04-28 02:55:03
2,ErgoSigmanauts,1251978,380358.285885,confirmed,1.000000,0.112525,50a3000448228642,27.001000,https://explorer.ergoplatform.com/en/blocks/58...,58be0effcc39dc7fc6bbc745acfd627ff3e30eab98668c...,9i8wsL9HYe4wRtuxXtnvki31uGJd6avKoQ79BXbz2sHWNZ...,ErgoSigmanauts,2024-04-27 07:59:45
3,ErgoSigmanauts,1251939,366775.443497,confirmed,1.000000,0.244779,502468dac3622052,27.018200,https://explorer.ergoplatform.com/en/blocks/a4...,a4dc120a4aa17097a163160eeb41c3dba31780f4086491...,9i3P4Ah9jp7nnSNMg6gxjXXHEWjkkBbPJR5RLRkMwEt3DZ...,ErgoSigmanauts,2024-04-27 06:54:45
4,ErgoSigmanauts,1251871,366775.443497,confirmed,1.000000,1.039931,50d8fd9c81187de2,27.008800,https://explorer.ergoplatform.com/en/blocks/2b...,2be39f033f12f55638a0e9a8755b7f96b73d15b948990c...,9fxwXaXp7x3i8gzfZ5zqMJJK1DbtyUeVGQL3mBaCg2cB8y...,ErgoSigmanauts,2024-04-27 04:31:16
5,ErgoSigmanauts,1251567,388383.156547,confirmed,1.000000,0.261117,555f000b343364e4,27.005100,https://explorer.ergoplatform.com/en/blocks/f8...,f826ef482455161ca689de707f3c239b3ca8a6e7360a2c...,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...,ErgoSigmanauts,2024-04-26 18:24:58
6,ErgoSigmanauts,1251499,388383.156547,confirmed,1.000000,6.396079,507971cae2ee4a18,27.004200,https://explorer.ergoplatform.com/en/blocks/c7...,c7c63f408c339ae4482914cbb8e93ad48b17defd20ec90...,9fHLdYBz5fPdDfMkGMfAsuj6EfXCpmM4GkamT23xeT3hzd...,ErgoSigmanauts,2024-04-26 16:00:40
7,ErgoSigmanauts,1249527,315887.249576,confirmed,1.000000,0.341527,50164a42c64cc4f1,27.015100,https://explorer.ergoplatform.com/en/blocks/4f...,4fea81e1a1a823f1c395a7d0757208381c9f6da4f68b83...,9fRQ8iCiCZekAewtjHmu7AmucWjvBTNUMeBFtnNkpfyWsN...,ErgoSigmanauts,2024-04-23 22:19:10
8,ErgoSigmanauts,1249440,315887.249576,confirmed,1.000000,0.019216,50187b62b18da535,27.010000,https://explorer.ergoplatform.com/en/blocks/22...,224c94abcd0ffa48553f6988c34bcb92c2490e76861f1c...,9f3FRr4XudxVs1V35At1X5yj7LmQmnWqG46LqFKVNRf2Tu...,ErgoSigmanauts,2024-04-23 19:17:03
9,ErgoSigmanauts,1249432,315887.249576,confirmed,1.000000,0.639227,5057d49b44b4ed21,27.012200,https://explorer.ergoplatform.com/en/blocks/ac...,acb0c18d072bc0fc0d0e69e95cb07ba996b53ef0ac669a...,9eZVqXVnrVWQKK19b7E7kp4ZyNqanp2z1mpKUJRaouNsme...,ErgoSigmanauts,2024-04-23 19:06:49


In [18]:
reader.get_miner_payment_stats('9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk')

{'Pending Shares': 350.037,
 'Pending Balance': 0.0,
 'Total Paid': 277.147,
 'Paid Today': 0,
 'Schema': 'PPLNS',
 'Price': 1.36,
 'Last Payment': '2024-04-27',
 'lastPaymentLink': 'https://explorer.ergoplatform.com/en/transactions/21ed45c3e6e33a2c8ff6320ad9cd82a2f5197463f54f1312ebac9449e0288e49'}

In [19]:
len('9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk')

51

In [20]:
payment_headers = ['pendingShares NUMERIC',
                    'pendingBalance NUMERIC',
                    'totalPaid NUMERIC',
                    'todayPaid NUMERIC',
                    'Schema VARCHAR(50)',
                    'Price NUMERIC',
                    'lastPayment VARCHAR(50)',
                    'lastPaymentLink TEXT',
                    'created_at TIMESTAMP',
                    'miner VARCHAR(100)']

live_worker_headers = ['worker VARCHAR(50)', 'hashrate NUMERIC', 'shares_per_second NUMERIC',
                       'created TIMESTAMP', 'miner VARCHAR(100)', 'effort NUMERIC',
                       'ttf NUMERIC', 'last_block_found VARCHAR(100)']

performance_headers = ['worker VARCHAR(50)', 'hashrate NUMERIC', 'shares_per_second NUMERIC',
                       'created TIMESTAMP', 'miner VARCHAR(60)']


In [21]:
def worker_to_df(data):
    rows = []
    for worker, details in data['workers'].items():
        row = {
            'worker': worker,
            'hashrate': round(details['hashrate'] / 1e6, 2), #MH/s
            'shares_per_second': round(details['sharesPerSecond'], 2),
            'created': format_datetime(data['created'])
        }
        rows.append(row)
    
    # Create DataFrame
    return pd.DataFrame(rows)

def insert_df_rows(df, table):
    for index, row in df.iterrows():
        database.insert_data(table, row.to_dict())


In [22]:
database = PostgreSQLDatabase('marctheshark', 'password', 'localhost', 5432, 'mining-db')
database.connect()
database.get_cursor()

database.delete_table('payment')
database.delete_table('live_worker')
database.delete_table('performance')
database.create_table('payment', payment_headers)
database.create_table('live_worker', live_worker_headers)
database.create_table('performance', performance_headers)

miner_data = reader.get_api_data('{}/{}'.format(reader.base_api, 'miners?pageSize=5000'))
miner_ls = [sample['miner'] for sample in miner_data]

time_now = pd.Timestamp.now()
stats = database.fetch_data('data')
block_data = database.fetch_data('block')
networkHashrate = stats['networkhashrate'][0] # use logic to get the lastest not just the first index
networkDifficulty = stats['networkdifficulty'][0]
for miner in miner_ls:
    # miner = '9hR5AWGMWcp9vvMvuiGNVzoiDfBztQqzBiK67iZwLCDWYLuUTaw'
    # miner = '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk'
    url = '{}/{}/{}'.format(reader.base_api, 'miners', miner)
    mining_data = reader.get_api_data(url)
    # print(mining_data.keys())

    
    payment_data = {k: v for k, v in mining_data.items() if k not in ['performance', 'performanceSamples']}
    payment_data['Schema'] = 'PPLNS'
    payment_data['Price'] = 2.0

    try:
        payment_data['lastPayment'] = mining_data['lastPayment'][:-17]
        payment_data['lastPaymentLink'] = mining_data['lastPaymentLink']
        
    except KeyError: 
        payment_data['lastPayment'] = 'N/A'
        payment_data['lastPaymentLink'] = 'Keep Mining!'

    except TypeError:
        payment_data['lastPayment'] = 'N/A'
        payment_data['lastPaymentLink'] = 'Keep Mining!'
    print(payment_data.keys())
    
    performance_samples = mining_data.pop('performanceSamples')
    
    payment_data['created_at'] = time_now
    payment_data['miner'] = miner
    # short_miner = '{}...{}'.format(data['miner'][:3], data['miner'][-5:])
    miner_blocks = block_data[block_data.miner == miner]
    
    performance_df = pd.concat(worker_to_df(sample) for sample in performance_samples)
    performance_df['miner'] = miner

    if miner_blocks.empty:
        # still need to adjust to pull from performance table for this miner
        latest = min(performance_df.created)
        last_block_found = 'N/A'

    else:
        latest = str(max(miner_blocks.time_found))
        last_block_found = latest

    try:
        live_performance = mining_data.pop('performance')
        live_df = worker_to_df(live_performance)
        live_df['miner'] = miner
        live_df['effort'] = [reader.calculate_mining_effort(networkDifficulty, networkHashrate,
                                                            temp_hash, latest) for temp_hash in live_df.hashrate]
        live_df['ttf'] = [reader.calculate_time_to_find_block(networkDifficulty, networkHashrate,
                                                              temp_hash) for temp_hash in live_df.hashrate]
        live_df['last_block_found'] = last_block_found
    
        insert_df_rows(live_df, 'live_worker') 
        print('live worker inserted')
        
    except KeyError:
        live_df = pd.DataFrame()
        print('no live data')
    
    database.insert_data('payment', payment_data)
    print('payments inserted')

    insert_df_rows(performance_df, 'performance') 
    print('performance inserted')
    # break


Table payment deleted successfully.
Table live_worker deleted successfully.
Table performance deleted successfully.


/home/whaleshark/Documents/sigmanaut-mining-pool-ui/utils/db_util.py:133: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, self.conn)


dict_keys(['pendingShares', 'pendingBalance', 'totalPaid', 'todayPaid', 'lastPayment', 'lastPaymentLink', 'Schema', 'Price'])
live worker inserted
payments inserted
performance inserted
dict_keys(['pendingShares', 'pendingBalance', 'totalPaid', 'todayPaid', 'lastPayment', 'lastPaymentLink', 'Schema', 'Price'])
live worker inserted
payments inserted
performance inserted
dict_keys(['pendingShares', 'pendingBalance', 'totalPaid', 'todayPaid', 'lastPayment', 'lastPaymentLink', 'Schema', 'Price'])
live worker inserted
payments inserted
performance inserted
dict_keys(['pendingShares', 'pendingBalance', 'totalPaid', 'todayPaid', 'lastPayment', 'lastPaymentLink', 'Schema', 'Price'])
live worker inserted
payments inserted
performance inserted
dict_keys(['pendingShares', 'pendingBalance', 'totalPaid', 'todayPaid', 'lastPayment', 'lastPaymentLink', 'Schema', 'Price'])
live worker inserted
payments inserted
performance inserted
dict_keys(['pendingShares', 'pendingBalance', 'totalPaid', 'todayPaid'

In [23]:
lw = database.fetch_data('live_worker')
lw

/home/whaleshark/Documents/sigmanaut-mining-pool-ui/utils/db_util.py:133: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, self.conn)


,worker,hashrate,shares_per_second,created,miner,effort,ttf,last_block_found
0,3950c56d,934.09,0.07,2024-04-28 04:20:34,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...,6.286,22.505,2024-04-26 18:24:58
1,6600xt_Rig,574.02,0.07,2024-04-28 04:20:34,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...,3.863,36.622,2024-04-26 18:24:58
2,8GPU_Rig,1176.15,0.10,2024-04-28 04:20:34,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...,7.914,17.873,2024-04-26 18:24:58
3,EpycDownstairs,570.60,0.07,2024-04-28 04:20:34,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...,3.840,36.842,2024-04-26 18:24:58
4,MinerDude_12x,759.69,0.07,2024-04-28 04:20:34,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...,5.112,27.672,2024-04-26 18:24:58
...,...,...,...,...,...,...,...,...
65,STINKY,199.33,0.04,2024-04-28 04:20:34,9fGzn8VCszvFMXo2KDrQiVowwSdchVT13ygX26RKUfkjcr...,1.002,105.463,N/A
66,qx3090,288.82,0.06,2024-04-28 04:20:34,9eZPTmn8zp5GJ7KZwTo8cEuxNdezWaY3hBbLeWid7EAZed...,27.961,72.785,2024-04-07 19:55:43
67,FubinbouErgoTrexMiner,145.26,0.03,2024-04-28 04:20:34,9iGCyowxwgf6cQef1RfEw6UCYoHJi6q1hZvKvcWyFY1kuL...,0.730,144.719,N/A
68,rustinmyeye,115.00,0.03,2024-04-28 04:20:34,9iQS22vfWdF2N84Lvv9jgAMFGLyL7t17SWbHGEfaddaG7Z...,23.469,182.799,2024-03-16 06:45:27


In [39]:
df = database.fetch_data('performance')
df.head()# need to create totals per created column for total hashrate plot

/home/whaleshark/Documents/sigmanaut-mining-pool-ui/utils/db_util.py:133: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, self.conn)


,worker,hashrate,shares_per_second,created,miner
0,3950c56d,759.34,0.06,2024-04-27 03:00:00,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...
1,6600xt_Rig,439.51,0.07,2024-04-27 03:00:00,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...
2,8GPU_Rig,1170.95,0.07,2024-04-27 03:00:00,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...
3,8GPU_Rig_3060ti,1252.93,0.07,2024-04-27 03:00:00,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...
4,EpycDownstairs,480.55,0.07,2024-04-27 03:00:00,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...


In [36]:
aggregated_df = df.groupby('created').agg({
    'hashrate': 'sum',                  # Sum of hashrate
    'shares_per_second': 'sum',         # Sum of shares_per_second
    'worker': 'nunique',                # Count of unique workers
    'miner': 'nunique'                  # Count of unique miners
})

In [38]:
aggregated_df['hashrate'] = aggregated_df['hashrate'] / 1e3 # converts MH/s to Gh/s
aggregated_df

,hashrate,shares_per_second,worker,miner
created,,,,
2024-04-27 03:00:00,49.14714,4.30,64,34
2024-04-27 04:00:00,50.53898,4.40,65,35
2024-04-27 05:00:00,51.38828,4.33,65,36
2024-04-27 06:00:00,51.85979,4.32,64,36
2024-04-27 07:00:00,52.23023,4.36,63,35
2024-04-27 08:00:00,51.43580,4.33,63,35
2024-04-27 09:00:00,52.07463,4.34,63,35
2024-04-27 10:00:00,51.23756,4.25,63,34
2024-04-27 11:00:00,52.39330,4.37,63,36


In [26]:
df = database.fetch_data('payment')
df

,pendingshares,pendingbalance,totalpaid,todaypaid,schema,price,lastpayment,lastpaymentlink,created_at,miner
0,299.336642,0.000000,42.151710,0.0,PPLNS,2.0,2024-04-27,https://explorer.ergoplatform.com/en/transacti...,2024-04-27 22:21:56.137801,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...
1,352.495713,0.000000,94.199954,0.0,PPLNS,2.0,2024-04-27,https://explorer.ergoplatform.com/en/transacti...,2024-04-27 22:21:56.137801,9f3FRr4XudxVs1V35At1X5yj7LmQmnWqG46LqFKVNRf2Tu...
2,350.036524,0.000000,277.146665,0.0,PPLNS,2.0,2024-04-27,https://explorer.ergoplatform.com/en/transacti...,2024-04-27 22:21:56.137801,9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPV...
3,292.384057,0.000000,64.881550,0.0,PPLNS,2.0,2024-04-27,https://explorer.ergoplatform.com/en/transacti...,2024-04-27 22:21:56.137801,9h6bjuzth7XDNeBxwFjpYSfmUkfBiepE8MGJcvmsVkeCtG...
4,243.026795,0.000000,204.967767,0.0,PPLNS,2.0,2024-04-27,https://explorer.ergoplatform.com/en/transacti...,2024-04-27 22:21:56.137801,9i3P4Ah9jp7nnSNMg6gxjXXHEWjkkBbPJR5RLRkMwEt3DZ...
5,235.835255,0.000000,55.838532,0.0,PPLNS,2.0,2024-04-27,https://explorer.ergoplatform.com/en/transacti...,2024-04-27 22:21:56.137801,9fHLdYBz5fPdDfMkGMfAsuj6EfXCpmM4GkamT23xeT3hzd...
6,254.089999,0.000000,198.602926,0.0,PPLNS,2.0,2024-04-27,https://explorer.ergoplatform.com/en/transacti...,2024-04-27 22:21:56.137801,9gXo8m2VyHvQ3FGvCg1GNWfyM5uX27BEKNCXDjQ6GiFsMJ...
7,52.433571,0.000000,17.395655,0.0,PPLNS,2.0,2024-04-26,https://explorer.ergoplatform.com/en/transacti...,2024-04-27 22:21:56.137801,9eZVqXVnrVWQKK19b7E7kp4ZyNqanp2z1mpKUJRaouNsme...
8,163.571680,0.000000,19.683209,0.0,PPLNS,2.0,2024-04-27,https://explorer.ergoplatform.com/en/transacti...,2024-04-27 22:21:56.137801,9fRQ8iCiCZekAewtjHmu7AmucWjvBTNUMeBFtnNkpfyWsN...
9,169.824307,0.000000,12.607252,0.0,PPLNS,2.0,2024-04-27,https://explorer.ergoplatform.com/en/transacti...,2024-04-27 22:21:56.137801,9fLYPigGHXkTyyQvU9zzoT3RTAXJ4dfHjbkg6ik2fHKKxj...


In [27]:
block = database.fetch_data('block')

/home/whaleshark/Documents/sigmanaut-mining-pool-ui/utils/db_util.py:133: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, self.conn)


In [28]:
block

,poolid,blockheight,networkdifficulty,status,confirmationprogress,effort,transactionconfirmationdata,reward,infolink,hash,miner,source,time_found
0,ErgoSigmanauts,1252559,422881.658950,pending,0.472222,0.017700,56dcc160776a8a71,27.022100,https://explorer.ergoplatform.com/en/blocks/f1...,f12b63d2743427539774bdc9f76bb207fa7a82b659cfa4...,9hSk13XDU8keWAp3KxKSgUVCkfDcw577jAjM6SgbRpuYeL...,ErgoSigmanauts,2024-04-28 03:06:52
1,ErgoSigmanauts,1252553,422881.658950,pending,0.555556,1.856882,50230ed620a8fa0b,27.000000,https://explorer.ergoplatform.com/en/blocks/9d...,9d7544bf89b699e7d31eaf49c4b8610d62d6545c1de9d2...,9i3P4Ah9jp7nnSNMg6gxjXXHEWjkkBbPJR5RLRkMwEt3DZ...,ErgoSigmanauts,2024-04-28 02:55:03
2,ErgoSigmanauts,1251978,380358.285885,confirmed,1.000000,0.112525,50a3000448228642,27.001000,https://explorer.ergoplatform.com/en/blocks/58...,58be0effcc39dc7fc6bbc745acfd627ff3e30eab98668c...,9i8wsL9HYe4wRtuxXtnvki31uGJd6avKoQ79BXbz2sHWNZ...,ErgoSigmanauts,2024-04-27 07:59:45
3,ErgoSigmanauts,1251939,366775.443497,confirmed,1.000000,0.244779,502468dac3622052,27.018200,https://explorer.ergoplatform.com/en/blocks/a4...,a4dc120a4aa17097a163160eeb41c3dba31780f4086491...,9i3P4Ah9jp7nnSNMg6gxjXXHEWjkkBbPJR5RLRkMwEt3DZ...,ErgoSigmanauts,2024-04-27 06:54:45
4,ErgoSigmanauts,1251871,366775.443497,confirmed,1.000000,1.039931,50d8fd9c81187de2,27.008800,https://explorer.ergoplatform.com/en/blocks/2b...,2be39f033f12f55638a0e9a8755b7f96b73d15b948990c...,9fxwXaXp7x3i8gzfZ5zqMJJK1DbtyUeVGQL3mBaCg2cB8y...,ErgoSigmanauts,2024-04-27 04:31:16
5,ErgoSigmanauts,1251567,388383.156547,confirmed,1.000000,0.261117,555f000b343364e4,27.005100,https://explorer.ergoplatform.com/en/blocks/f8...,f826ef482455161ca689de707f3c239b3ca8a6e7360a2c...,9g4f585vPtgA5PKhSRcaPBhtfW3HYRw2qe3aemxxnD5Y1K...,ErgoSigmanauts,2024-04-26 18:24:58
6,ErgoSigmanauts,1251499,388383.156547,confirmed,1.000000,6.396079,507971cae2ee4a18,27.004200,https://explorer.ergoplatform.com/en/blocks/c7...,c7c63f408c339ae4482914cbb8e93ad48b17defd20ec90...,9fHLdYBz5fPdDfMkGMfAsuj6EfXCpmM4GkamT23xeT3hzd...,ErgoSigmanauts,2024-04-26 16:00:40
7,ErgoSigmanauts,1249527,315887.249576,confirmed,1.000000,0.341527,50164a42c64cc4f1,27.015100,https://explorer.ergoplatform.com/en/blocks/4f...,4fea81e1a1a823f1c395a7d0757208381c9f6da4f68b83...,9fRQ8iCiCZekAewtjHmu7AmucWjvBTNUMeBFtnNkpfyWsN...,ErgoSigmanauts,2024-04-23 22:19:10
8,ErgoSigmanauts,1249440,315887.249576,confirmed,1.000000,0.019216,50187b62b18da535,27.010000,https://explorer.ergoplatform.com/en/blocks/22...,224c94abcd0ffa48553f6988c34bcb92c2490e76861f1c...,9f3FRr4XudxVs1V35At1X5yj7LmQmnWqG46LqFKVNRf2Tu...,ErgoSigmanauts,2024-04-23 19:17:03
9,ErgoSigmanauts,1249432,315887.249576,confirmed,1.000000,0.639227,5057d49b44b4ed21,27.012200,https://explorer.ergoplatform.com/en/blocks/ac...,acb0c18d072bc0fc0d0e69e95cb07ba996b53ef0ac669a...,9eZVqXVnrVWQKK19b7E7kp4ZyNqanp2z1mpKUJRaouNsme...,ErgoSigmanauts,2024-04-23 19:06:49


In [29]:
block.effort.sum() / len(block.effort)

0.9001368828200207

In [30]:
block['effort'].expanding().mean()

0     0.017700
1     0.937291
2     0.662369
3     0.557971
4     0.654363
5     0.588822
6     1.418430
7     1.283817
8     1.143306
9     1.092898
10    1.108310
11    1.045226
12    1.017586
13    1.016472
14    0.959454
15    0.905421
16    0.892766
17    1.012050
18    0.986956
19    0.939274
20    0.897986
21    0.951897
22    0.915587
23    0.892494
24    0.872178
25    0.890993
26    0.869255
27    0.913538
28    0.895280
29    0.881116
30    0.869378
31    0.885790
32    0.918964
33    0.896527
34    0.875991
35    0.862191
36    0.873222
37    0.866254
38    0.875862
39    0.870861
40    0.862432
41    0.845923
42    0.834356
43    0.859186
44    0.840669
45    0.827552
46    0.831493
47    0.823895
48    0.813804
49    0.798139
50    0.788121
51    0.788299
52    0.863104
53    0.863704
54    0.900137
Name: effort, dtype: float64